In [ ]:
def getRateTime(row):
	uIdx, mIdx, rate, time = row
	rate = float(rate)
	#60 seconds * 60 minutes * 24 hours * 7days, 52 weeks * 25 years since 1970
	time = int(time) // 604800 - 1300
	return (rate, time)

def getRate(rateTime):
	rate, time = rateTime
	return rate

def getTime(rateTime):
	rate, time = rateTime
	return time

class regression(object):
	def __init__(self, mRate, mTime):
		self.mRate = mRate
		self.mTime = mTime
		return

	def getK(self, rateTime):
		rate, time = rateTime
		dRate = rate - self.mRate
		dTime = time - self.mTime
		return (dRate * dTime, dTime ** 2)

	def getB(self, k):
		return self.mRate - self.mTime * k

def sumK(a, b):
	axy, ax2 = a
	bxy, bx2 = b
	return (axy + bxy, ax2 + bx2)

class rescale(object):
	def __init__(self, k, b):
		self.k = k
		self.b = b
		return

	def scale(self, row):
		uIdx, mIdx, rate, time = row
		rate = float(rate)
		sTime = int(time) // 604800 - 1300
		sRate = rate - (self.k * sTime + self.b)
		return (uIdx, mIdx, sRate, time)

In [ ]:
rateData = spark.read.csv('/user/hz333/data/project/train.csv', header = False)

In [ ]:
#(uIdx, mIdx, rate, time) => (rate, time)
rateTime = rateData.rdd.map(getRateTime)

#(rate, time) => (rate, )
rate = rateTime.map(getRate)
#(rate, time) => (time, )
time = rateTime.map(getTime)
meanRate = rate.mean()
meanTime = time.mean()

reg = regression(meanRate, meanTime)

#(rate, time) => ((rate - mRate) * (time - mTime), (time - mTime) ** 2)
kData = rateTime.map(reg.getK)
#((rate - mRate) * (time - mTime), (time - mTime) ** 2) => (numerator, denominator)
kData = kData.reduce(sumK)

k = kData[0] / kData[1]
b = reg.getB(k) - 3.25 #move rates in range [0, 5]

In [ ]:
print(k)
print(b)

In [ ]:
res = rescale(k, b)
#(uIdx, mIdx, rate, time) => (uIdx, mIdx, sRate, time)
sRateData = rateData.rdd.map(res.scale)
sRateData.take(10)

In [ ]:
minSRate = sRateData.min(lambda x: x[2])
maxSRate = sRateData.max(lambda x: x[2])

In [ ]:
print(minSRate)
print(maxSRate)

In [ ]:
sTrainCSV = spark.createDataFrame(sRateData, samplingRatio = 1)
sTrainCSV.repartition(1).write.option('header', 'false').csv('/user/hz333/data/project/sTrain.csv')

In [ ]:
testData = spark.read.csv('/user/hz333/data/project/test.csv', header = False)
sRateData = testData.rdd.map(res.scale)
print(sRateData.take(10))
minSRate = sRateData.min(lambda x: x[2])
maxSRate = sRateData.max(lambda x: x[2])
print(minSRate)
print(maxSRate)
sTestCSV = spark.createDataFrame(sRateData, samplingRatio = 1)
sTestCSV.repartition(1).write.option('header', 'false').csv('/user/hz333/data/project/sTest.csv')

In [ ]:
validData = spark.read.csv('/user/hz333/data/project/valid.csv', header = False)
sRateData = validData.rdd.map(res.scale)
print(sRateData.take(10))
minSRate = sRateData.min(lambda x: x[2])
maxSRate = sRateData.max(lambda x: x[2])
print(minSRate)
print(maxSRate)
sValidCSV = spark.createDataFrame(sRateData, samplingRatio = 1)
sValidCSV.repartition(1).write.option('header', 'false').csv('/user/hz333/data/project/sValid.csv')